# Выбор локации для скважины

Нужно решить, где бурить новую скважину. Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описание данных

id — уникальный идентификатор скважины;<br/>
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);<br/>
product — объём запасов в скважине (тыс. баррелей).<br/>

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [2]:
region_1 = pd.read_csv('./geo_data_0.csv')
region_2 = pd.read_csv('./geo_data_1.csv')
region_3 = pd.read_csv('./geo_data_2.csv')

In [3]:
#информация о таблице region_1 (пропусков нет, типы данных соответствующие)
region_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
#удалим столбец id, так как он не содержит важных данных
region_1 = region_1.drop('id',axis=1)

In [5]:
#проверка на дубли
region_1.duplicated().sum()

0

In [6]:
#информация о таблице region_2 (пропусков нет, типы данных соответствующие)
region_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
#удалим столбец id, так как он не содержит важных данных
region_2 = region_2.drop('id',axis=1)

In [8]:
#проверка на дубли
region_2.duplicated().sum()

0

In [9]:
#информация о таблице region_3 (пропусков нет, типы данных соответствующие)
region_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
#удалим столбец id, так как он не содержит важных данных
region_3 = region_3.drop('id',axis=1)

In [11]:
#проверка на дубли
region_3.duplicated().sum()

0

### Вывод:

Посмотрели информацию о каждом регионе: пропусков нет, дубликатов нет, тип данных соответствующий, удалили столбец с данными 'id', так как в этом столбце содержится не нужная для нас информация, которая в дальнейшем никак не повлияет на изучение скважин.

## Обучение и проверка модели

Напишем функцию, где выбираем признаки, разбиваем данные, строим модель Линейной регрессии и 
находим средний запас предсказанного сырья, RMSE

In [12]:
def split_model(data,name):
    features = data.drop('product', axis=1)
    target = data['product']

    features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)
                                                                           
    model = LinearRegression().fit(features_train,target_train)
    predicted = model.predict(features_valid)
    predicted = pd.Series(predicted)
    rmse = mean_squared_error(target_valid,predicted) ** 0.5
    average_stock = sum(predicted) / len(predicted)
    
    print(name)
    print('RMSE: {:.2f}'. format(rmse))
    print('Cредний запас предсказанного сырья: {:.2f}'. format(average_stock))
    
    return predicted, target_valid.reset_index(drop=True), rmse

In [13]:
predicted_1, validation_1, rmse_1 = split_model(region_1,'Регион 1')
predicted_2, validation_2, rmse_2 = split_model(region_2,'Регион 2')
predicted_3, validation_3, rmse_3 = split_model(region_3,'Регион 3')

Регион 1
RMSE: 37.58
Cредний запас предсказанного сырья: 92.59
Регион 2
RMSE: 0.89
Cредний запас предсказанного сырья: 68.73
Регион 3
RMSE: 40.03
Cредний запас предсказанного сырья: 94.97


In [14]:
#посмотрим объем запасов в скважине второго региона
region_2['product'].value_counts()

53.906522     8472
26.953261     8468
84.038886     8431
57.085625     8390
3.179103      8337
80.859783     8320
30.132364     8306
134.766305    8304
110.992147    8303
0.000000      8235
137.945408    8233
107.813044    8201
Name: product, dtype: int64

#### Вывод:

   Самый лучший регион по запасам нефти оказался второй (так как RMSE=0.89), потом регион 1 (RMSE = 37.58) и регион 3 (RMSE = 40). Но средний запас предсказанного сырья больше всего у 1 и 3 региона. Данные второго региона повторяются, возможно некорректно были введены, нужно отдать на перепроверку <br/> 

## Подготовка к расчёту прибыли

In [15]:
#ключевые значения
best_points = 200
budget = 10**10
unit_income_barel = 450000

Посчитаем точку безубыточности и среднюю прибыль на лучшую точку

In [16]:
bezubyt_point = budget / unit_income_barel
print('Точка безубыточности(средний объем нефти на регион): {:.2f} тыс.баррелей.'. format(bezubyt_point))

average_profit_per_point = bezubyt_point / best_points
print('Достаточный объём сырья для безубыточной разработки новой скважины: {:.2f} тыс.баррелей.'. format(average_profit_per_point))

Точка безубыточности(средний объем нефти на регион): 22222.22 тыс.баррелей.
Достаточный объём сырья для безубыточной разработки новой скважины: 111.11 тыс.баррелей.


In [17]:
print('Средний запас нефти на скважину в регионе 1: ', region_1['product'].mean())
print('Средний запас нефти на скважину в регионе 2: ', region_2['product'].mean())
print('Средний запас нефти на скважину в регионе 3: ', region_3['product'].mean())

Средний запас нефти на скважину в регионе 1:  92.49999999999976
Средний запас нефти на скважину в регионе 2:  68.82500000002561
Средний запас нефти на скважину в регионе 3:  95.00000000000041


#### Вывод:

Мы выяснили, что точка безубыточности равна 22 222 тыс.баррелей. Достаточный объём сырья для безубыточной разработки новой скважины: 111.11 тыс.баррелей. 
Для первого и третьего региона больше вероятность окупить расходы, так как средний запас в регионах 92.5 и 95, что нельзя     сказать о регионе 2.

## Расчёт прибыли и рисков 

In [18]:
#посчитали прибыль(отсортировали по убыванию и выбрали 200 лучших точек)
def profit(predicted, target):
    sort_pred = predicted.sort_values(ascending=False)
    top_targ = target[sort_pred.index][:200]
    revenue = top_targ.sum() * unit_income_barel
    return revenue - budget

Применим технику Bootstrap с 1000 выборок, найдем доверительны интервал, рассчитаем среднюю прибыль и риски для каждого региона:

In [19]:
state = np.random.RandomState(12345)

def bootstrap(predicted, target):
        values = []
        for i in range(1000):
            target_subsample = target.sample(n=500, replace=True, random_state=state)
            predicted_subsample = predicted[target_subsample.index]
            values.append(profit(predicted_subsample, target))

        values = pd.Series(values)
        lower = values.quantile(0.025)
        upper = values.quantile(0.975)
        aver = values.mean()
        risk = (values < 0).mean()
        return aver, lower, upper, risk

In [20]:
i = 1
for preds, target in [[predicted_1,validation_1], [predicted_2,validation_2], [predicted_3,validation_3]]:
    aver, lower, upper, risk = bootstrap(preds, target)
    print('Регион', i)
    print('----------------------------')
    print('Средняя прибыль: {:.2f} тыс.руб.'. format(aver))
    print('95%-й доверительный интервал: от {:.2f} до {:.2f} тыс.руб.'.format(lower, upper))
    print('Риск убытков: {:.2%}'. format(risk))
    print()
    i+=1

Регион 1
----------------------------
Средняя прибыль: 396164984.80 тыс.руб.
95%-й доверительный интервал: от -111215545.89 до 909766941.55 тыс.руб.
Риск убытков: 6.90%

Регион 2
----------------------------
Средняя прибыль: 461155817.28 тыс.руб.
95%-й доверительный интервал: от 78050810.75 до 862952060.26 тыс.руб.
Риск убытков: 0.70%

Регион 3
----------------------------
Средняя прибыль: 392950475.17 тыс.руб.
95%-й доверительный интервал: от -112227625.38 до 934562914.55 тыс.руб.
Риск убытков: 6.50%



#### Вывод:
 
С помощью бутстрэпа нашли доверительный интервал, так же посчитали среднюю прибыль и риски для каждого региона.

Мой выбор остановился на втором регионе, так как он оказался менее рискованным (0.7%), чем остальные(6.90% и 6.50%), плюс средняя прибыль (461155817.28 тыс.руб) больше. Если сравнивать доверительный интервал, то здесь так же больше доверия вызывает регион под номером 2.